(numbers)=
# Numbers

## Introduction

In this chapter, you'll learn useful tools for creating and manipulating numeric vectors. We'll start by going into a little more detail of `.count()` before diving into various numeric transformations. You'll then learn about more general transformations that can be applied to other types of column, but are often used with numeric column. Then you'll learn about a few more useful summaries.

### Prerequisites

This chapter mostly uses functions from **pandas**, which you are likely to already have installed bu you can install using `pip install pandas` in the terminal. We'll use real examples from nycflights13, as well as toy examples made with fake data.

Let's first load up the NYC flights data


In [ ]:
import matplotlib_inline.backend_inline
import matplotlib.pyplot as plt

# Plot settings
plt.style.use(
    "https://github.com/aeturrell/coding-for-economists/raw/main/plot_style.txt"
)
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/byuidatascience/data4python4ds/master/data-raw/flights/flights.csv"
flights = pd.read_csv(url)

### Counts

It's surprising how much data science you can do with just counts and a little basic arithmetic, so **pandas** strives to make counting as easy as possible with `.count()` and `.value_counts()`. The former just provides a straight count of all the non NA items:

In [ ]:
flights["dest"].count()

The latter provides a count broken down by type:

In [ ]:
flights["dest"].value_counts()

This is automatically sorted in order of the most common category. You can perform the same computation "by hand" with `group_by()`, `agg()` and then using the count function. This is useful because it allows you to compute other summaries at the same time:

In [ ]:
(
    flights.groupby(["dest"])
    .agg(
        mean_delay=("dep_delay", "mean"),
        count_flights=("dest", "count"),
    )
    .sort_values(by="count_flights", ascending=False)
)

Note that a weighted count is just a sum. For example you could "count" the number of miles each plane flew:

In [ ]:
(flights.groupby("tailnum").agg(miles=("distance", "sum")))

You can count missing values by combining `sum()` and `isnull()`. In the flights dataset this represents flights that are cancelled. Note that because there isn't a simple string name for applying `.isnull` followed by `.sum` (unlike just running `sum`, which would be given by the string "sum"), we need to use a lambda function in the below:

In [ ]:
(flights.groupby("dest").agg(n_cancelled=("dep_time", lambda x: x.isnull().sum())))

## Numeric Transformations

Transformation functions have an output is the same length as the input. The vast majority of transformation functions are either built into Python or come with the numerical package **numpy**. It's impractical to list all the possible numerical transformations so this section will show the most useful ones.

Basic number arithmatic is achieved by `+` (addition), `-` (subtraction), `*` (multiplication), `/` (division), `**` (powers), `%` (modulo), and `@` (tensor product). Most of these functions don't need a huge amount of explanation because you'll be familiar with them already (and you can look up the others when you do need them).

When you have two numeric columns of equal length and you add or subtract them, it's pretty obvious what's going to happen. But we do need to talk about what happens when there is a variable involved that is *not* as long as the column. This is important for operations like `flights.assign(air_time = air_time / 60)` because there are 336,776 numbers on the left of `/` but only one on the right. In this case, **pandas** will understand that you'd like to divide *all* values of air time by 60. This is sometimes called 'broadcasting'. Below is a digram that tries to explain what's going on:

![](https://numpy.org/doc/stable/_images/broadcasting_1.png)

You can find out much more about [broadcasting on the **numpy** documentation](https://numpy.org/doc/stable/user/basics.broadcasting.html). **pandas** is built on top of **numpy** and inherits some of its functionality.  


When operating on two columns, **pandas** compares their shapes element-wise. Two columns are compatible when they are equal, or one of them is a scalar. If these conditions are not met, you will get an error.



### Minimum and Maximum

The arithmetic functions do what you'd expect.

In [ ]:
flights["distance"].max()

Sometimes, you'd like to look at the maximum or minimum value across rows *or* columns. As often is the case with **pandas**, you can specify rows or columns to apply functions to by passing `axis=0` (index) or `axis=1` (columns) to that function. The axis designation can be confusing: remember that you are asking which dimension you wish to aggregate over, leaving you with the other dimension. So if we wish to find the minimum in each row, we aggregate / collapse columns, so we need to pass `axis=1`.

In [ ]:
df = pd.DataFrame({"x": [1, 5, 7], "y": [3, 2, pd.NA]})
df

Now let's find the min by row:

In [ ]:
df.min(axis=1)

### Modular arithmetic

Modular arithmetic is the technical name for the type of math you do on whole numbers, i.e. division that yields a whole number and a remainder. In Python, `//` does integer division and `%` computes the remainder:

In [ ]:
print([x for x in range(1, 11)])
print("divided by 3 gives")
print("remainder:")
print([x % 3 for x in range(1, 11)])
print("divisions:")
print([x // 3 for x in range(1, 11)])

Modular arithmetic is handy for the flights dataset, because we can use it to unpack the `sched_dep_time` variable into and `hour` and `minute`:

In [ ]:
flights.assign(
    hour=lambda x: x["sched_dep_time"] // 100,
    minute=lambda x: x["sched_dep_time"] % 100,
)

We can combine that with the `mean(is.na(x))` trick from @sec-logical-summaries to see how the proportion of cancelled flights varies over the course of the day. [TODO]

### Logarithms

Logarithms are an incredibly useful transformation for dealing with data that ranges across multiple orders of magnitude.
They also convert exponential growth to linear growth. For example, take compounding interest --- the amount of money you have at `year + 1` is the amount of money you had at `year` multiplied by the interest rate. That gives a formula like `money = starting * interest ** year`:

```{r}
starting <- 100
interest <- 1.05
money <- tibble(
  year = 2000 + 1:50,
  money = starting * interest^(1:50)
)
```


In [ ]:
import numpy as np

starting = 100
interest = 1.05
money = pd.DataFrame(
    {"year": 2000 + np.arange(1, 51), "money": starting * interest ** np.arange(1, 51)}
)
money.head()

If you plot this data, you'll get an exponential curve:

In [ ]:
money.plot(x="year", y="money");

Log transforming the y-axis gives a straight line:

In [ ]:
money.plot(x="year", y="money", logy=True);

This a straight line because `log(money) = log(starting) + n * log(interest)` matches the pattern for a line, `y = m * x + b`. This is a useful pattern: if you see a (roughly) straight line after log-transforming the y-axis, you know that there's underlying exponential growth.

If you're log-transforming your data you have a choice of a lot of logarithms provided by **numpy** but there are three ones you'll want to commonly use: assuming you've imported it using `import numpy as np`, you have `np.log()` (the natural log, base e), `np.log2()` (base 2), and `np.log10()` (base 10).

The inverse of `log()` is `np.exp()`; to compute the inverse of `np.log2()` or `np.log10()` you'll need to use `2**` or `10**`.